In [3]:
import pdfplumber

with pdfplumber.open("../data/raw/kotak-bankstatement.pdf") as pdf:
    for page in pdf.pages:
        print(page.extract_text())

ADITYA ANIL GUJAR Period : 01-04-2024 to 31-03-2025
Cust Reln No : 468666680
H-501, PRISTINE PROLIFE Account No : 6845525302
DATTA MANDIR ROAD Currency : INR
WAKAD Branch : PUNE-HINJEWADI
PUNE-411057 Nominee Registered : Y
MAHARASHTRA,INDIA Branch Address : WHITE SQUARE
HINJEWADI, WAKAD ROAD, PUNE
PUNE-411057
MAHARASHTRA,INDIA
Branch Phone No. : 9764443663
MICR Code : 411485028
IFSC Code : KKBK0001768
Withdrawal(Dr)/
Date Narration Chq/Ref No Balance
Deposit(Cr)
01-04-2024 NACH-MUT-DR-GROWW PAY SERVICES P- NACHDR01042400128740 100.00(Dr) 301.60(Cr)
IS47GA8EAUUE
02-04-2024 UPI/Premsagar super/409326729134/UPI UPI-409309358754 210.00(Dr) 91.60(Cr)
03-04-2024 UPI/ADITYA ANIL GUJ/409403199750/UPI UPI-409489026299 600.00(Cr) 691.60(Cr)
06-04-2024 NEFT CMS4072835070 NEXTBILLION NEFTINW-0820969933 2,338.74(Cr) 3,030.34(Cr)
TECHNOLOGY PRIVATE
14-04-2024 UPI/Alamgir Alam/410575365473/UPI UPI-410552075137 800.00(Dr) 2,230.34(Cr)
14-04-2024 UPI/Premsagar super/410557551761/UPI UPI-410586916148 70

In [4]:
pdf.pages

[<Page:1>,
 <Page:2>,
 <Page:3>,
 <Page:4>,
 <Page:5>,
 <Page:6>,
 <Page:7>,
 <Page:8>,
 <Page:9>,
 <Page:10>,
 <Page:11>,
 <Page:12>,
 <Page:13>,
 <Page:14>,
 <Page:15>,
 <Page:16>,
 <Page:17>,
 <Page:18>,
 <Page:19>,
 <Page:20>,
 <Page:21>,
 <Page:22>,
 <Page:23>,
 <Page:24>,
 <Page:25>,
 <Page:26>]

In [5]:
import pdfplumber

# Open the PDF file
with pdfplumber.open("../data/raw/kotak-bankstatement.pdf") as pdf:
    print(f"Total pages: {len(pdf.pages)}")
    first_page = pdf.pages[0]

Total pages: 26


In [7]:
import pdfplumber

with pdfplumber.open("../data/raw/kotak-bankstatement.pdf") as pdf:
    first_page = pdf.pages[0]
    tables = first_page.extract_tables()
    for table in tables:
        for row in table:
            print(row)  # <-- Use it inside the 'with' block

In [8]:
import pdfplumber

with pdfplumber.open("../data/raw/kotak-bankstatement.pdf") as pdf:
    print(f"Total pages: {len(pdf.pages)}")
    first_page = pdf.pages[0]
    
    tables = first_page.extract_tables()
    print(f"Number of tables found: {len(tables)}")

    for i, table in enumerate(tables):
        print(f"\nTable {i+1}:")
        for row in table:
            print(row)

Total pages: 26
Number of tables found: 0


In [9]:
import pdfplumber
import pandas as pd
import re

def parse_kotak_bank_statement(pdf_path):
    transactions = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue

            lines = text.split('\n')

            for line in lines:
                # Match lines like: 01-04-2024 UPI/... 100.00(Dr) 301.60(Cr)
                match = re.match(r"^(\d{2}-\d{2}-\d{4}) (.+?)\s+([A-Z0-9\-]+)?\s*([\d,]+\.\d{2})\((Dr|Cr)\)\s+([\d,]+\.\d{2})\((Dr|Cr)\)", line)
                if match:
                    date = match.group(1)
                    narration = match.group(2).strip()
                    reference = match.group(3) if match.group(3) else ''
                    
                    amt1 = match.group(4).replace(',', '')
                    type1 = match.group(5)
                    amt2 = match.group(6).replace(',', '')
                    type2 = match.group(7)

                    # Determine debit and credit based on tags
                    debit = float(amt1) if type1 == 'Dr' else (float(amt2) if type2 == 'Dr' else 0.0)
                    credit = float(amt1) if type1 == 'Cr' else (float(amt2) if type2 == 'Cr' else 0.0)
                    balance = float(amt2) if type2 == 'Cr' else float(amt1)

                    transactions.append({
                        'DATE': date,
                        'NARRATION': narration,
                        'REFERENCE': reference,
                        'DEBIT': debit,
                        'CREDIT': credit,
                        'BALANCE': balance
                    })

    df = pd.DataFrame(transactions)
    print(f"✅ Extracted {len(df)} transactions")
    return df

# Example usage:
if __name__ == "__main__":
    df = parse_kotak_bank_statement("../data/raw/kotak-bankstatement.PDF")
    df.to_csv("../data/processed/kotak_transactions.csv", index=False)
    print("💾 Saved to kotak_transactions.csv")
    print(df.head())

✅ Extracted 937 transactions
💾 Saved to kotak_transactions.csv
         DATE                             NARRATION             REFERENCE  \
0  01-04-2024     NACH-MUT-DR-GROWW PAY SERVICES P-  NACHDR01042400128740   
1  02-04-2024  UPI/Premsagar super/409326729134/UPI      UPI-409309358754   
2  03-04-2024  UPI/ADITYA ANIL GUJ/409403199750/UPI      UPI-409489026299   
3  06-04-2024        NEFT CMS4072835070 NEXTBILLION    NEFTINW-0820969933   
4  14-04-2024     UPI/Alamgir Alam/410575365473/UPI      UPI-410552075137   

   DEBIT   CREDIT  BALANCE  
0  100.0   301.60   301.60  
1  210.0    91.60    91.60  
2    0.0   600.00   691.60  
3    0.0  2338.74  3030.34  
4  800.0  2230.34  2230.34  
